# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4fb015b430>
├── 'a' --> tensor([[ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489]])
└── 'x' --> <FastTreeValue 0x7f4fb0155220>
    └── 'c' --> tensor([[-0.9313,  0.0848,  1.0034,  0.1700],
                        [ 1.4125, -1.1504, -0.0371,  0.8486],
                        [-0.9554, -0.1132, -0.8440,  0.5613]])

In [4]:
t.a

tensor([[ 1.8482, -0.2578, -0.3141],
        [-0.7162, -0.0363, -0.3489]])

In [5]:
%timeit t.a

64.6 ns ± 0.0482 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4fb015b430>
├── 'a' --> tensor([[ 0.5165, -0.2982, -0.9350],
│                   [-0.7668,  1.7618, -0.4617]])
└── 'x' --> <FastTreeValue 0x7f4fb0155220>
    └── 'c' --> tensor([[-0.9313,  0.0848,  1.0034,  0.1700],
                        [ 1.4125, -1.1504, -0.0371,  0.8486],
                        [-0.9554, -0.1132, -0.8440,  0.5613]])

In [7]:
%timeit t.a = new_value

75.6 ns ± 0.0526 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.8482, -0.2578, -0.3141],
               [-0.7162, -0.0363, -0.3489]]),
    x: Batch(
           c: tensor([[-0.9313,  0.0848,  1.0034,  0.1700],
                      [ 1.4125, -1.1504, -0.0371,  0.8486],
                      [-0.9554, -0.1132, -0.8440,  0.5613]]),
       ),
)

In [10]:
b.a

tensor([[ 1.8482, -0.2578, -0.3141],
        [-0.7162, -0.0363, -0.3489]])

In [11]:
%timeit b.a

57.9 ns ± 0.0363 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0190, -0.9133,  0.6458],
               [-0.7256,  1.6573, -0.5043]]),
    x: Batch(
           c: tensor([[-0.9313,  0.0848,  1.0034,  0.1700],
                      [ 1.4125, -1.1504, -0.0371,  0.8486],
                      [-0.9554, -0.1132, -0.8440,  0.5613]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.101 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

860 ns ± 0.507 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

148 µs ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

148 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4f84590640>
├── 'a' --> tensor([[[ 1.8482, -0.2578, -0.3141],
│                    [-0.7162, -0.0363, -0.3489]],
│           
│                   [[ 1.8482, -0.2578, -0.3141],
│                    [-0.7162, -0.0363, -0.3489]],
│           
│                   [[ 1.8482, -0.2578, -0.3141],
│                    [-0.7162, -0.0363, -0.3489]],
│           
│                   [[ 1.8482, -0.2578, -0.3141],
│                    [-0.7162, -0.0363, -0.3489]],
│           
│                   [[ 1.8482, -0.2578, -0.3141],
│                    [-0.7162, -0.0363, -0.3489]],
│           
│                   [[ 1.8482, -0.2578, -0.3141],
│                    [-0.7162, -0.0363, -0.3489]],
│           
│                   [[ 1.8482, -0.2578, -0.3141],
│                    [-0.7162, -0.0363, -0.3489]],
│           
│                   [[ 1.8482, -0.2578, -0.3141],
│                    [-0.7162, -0.0363, -0.3489]]])
└── 'x' --> <FastTreeValue 0x7f4f845904f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 54.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4f84590280>
├── 'a' --> tensor([[ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489],
│                   [ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489],
│                   [ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489],
│                   [ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489],
│                   [ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489],
│                   [ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489],
│                   [ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489],
│                   [ 1.8482, -0.2578, -0.3141],
│                   [-0.7162, -0.0363, -0.3489]])
└── 'x' --> <FastTreeValue 0x7f4f02e6b340>
    └── 'c' --> tensor([[-0.9313,  0.0848,  1.0034,  0.1700],
                        [ 1.4125, -1.1504, -0.0371,  0.8486],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.1 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.9313,  0.0848,  1.0034,  0.1700],
                       [ 1.4125, -1.1504, -0.0371,  0.8486],
                       [-0.9554, -0.1132, -0.8440,  0.5613]],
              
                      [[-0.9313,  0.0848,  1.0034,  0.1700],
                       [ 1.4125, -1.1504, -0.0371,  0.8486],
                       [-0.9554, -0.1132, -0.8440,  0.5613]],
              
                      [[-0.9313,  0.0848,  1.0034,  0.1700],
                       [ 1.4125, -1.1504, -0.0371,  0.8486],
                       [-0.9554, -0.1132, -0.8440,  0.5613]],
              
                      [[-0.9313,  0.0848,  1.0034,  0.1700],
                       [ 1.4125, -1.1504, -0.0371,  0.8486],
                       [-0.9554, -0.1132, -0.8440,  0.5613]],
              
                      [[-0.9313,  0.0848,  1.0034,  0.1700],
                       [ 1.4125, -1.1504, -0.0371,  0.8486],
                       [-0.9554, -0.1132, -0.8440,  0.5613]],

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.9313,  0.0848,  1.0034,  0.1700],
                      [ 1.4125, -1.1504, -0.0371,  0.8486],
                      [-0.9554, -0.1132, -0.8440,  0.5613],
                      [-0.9313,  0.0848,  1.0034,  0.1700],
                      [ 1.4125, -1.1504, -0.0371,  0.8486],
                      [-0.9554, -0.1132, -0.8440,  0.5613],
                      [-0.9313,  0.0848,  1.0034,  0.1700],
                      [ 1.4125, -1.1504, -0.0371,  0.8486],
                      [-0.9554, -0.1132, -0.8440,  0.5613],
                      [-0.9313,  0.0848,  1.0034,  0.1700],
                      [ 1.4125, -1.1504, -0.0371,  0.8486],
                      [-0.9554, -0.1132, -0.8440,  0.5613],
                      [-0.9313,  0.0848,  1.0034,  0.1700],
                      [ 1.4125, -1.1504, -0.0371,  0.8486],
                      [-0.9554, -0.1132, -0.8440,  0.5613],
                      [-0.9313,  0.0848,  1.0034,  0.1700],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

332 µs ± 1.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
